In [3]:
import sys
sys.path.append('/thesis')

In [4]:
import json
import pickle
# import logging
# import time

import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt

from helper.character_encoder import DictionaryCharacterEncoder
from helper.prediction import predict_sequence

from implementation.seq2seq.transformer.utils import masked_loss, masked_accuracy
from implementation.seq2seq.transformer.keras_nlp import prepare_batches
from implementation.seq2seq.transformer.keras_nlp import construct_model_w_teacher_forcing

import tensorflow as tf
import keras_nlp

tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
# Input file paths
jrc_file = '/thesis/data/jrc_person_pairs.csv'
wikidata_file = '/thesis/data/wikidata_person_pairs.csv'

model_serialization_path = '/thesis/models/transformer_mixed_kerasnlp_1'

## Loading & Preprocessing

In [6]:
# Preprocessing
MAX_SEQ_LENGTH = 30  # 40
NUM_SAMPLES = 550000
VALIDATION_SPLIT = 0.3
RANDOM_STATE = 1010
BATCH_SIZE = 64

# Model
EMBEDDING_DIM = 64
INTERMEDIATE_DIM = 512
NUM_ENCODER_HEADS = 8
NUM_DECODER_HEADS = 8
NUM_ENCODER_LAYERS = 2
NUM_DECODER_LAYERS = 4

DROPOUT = 0.2

# Training
EPOCHS = 60
CHECKPOINT_FREQ = 10

In [7]:
pairs_df = pd.read_csv(jrc_file, sep='|', encoding='utf-8')[['input', 'target']]
pairs_df = pairs_df[(pairs_df['input'].str.len() <= MAX_SEQ_LENGTH) & (pairs_df['target'].str.len() <= MAX_SEQ_LENGTH)]
print('Number of JRC pairs:', len(pairs_df))
pairs_df2 = pd.read_csv(wikidata_file, sep='|', encoding='utf-8')[['input', 'target']]
pairs_df2 = pairs_df2[(pairs_df2['input'].str.len() <= MAX_SEQ_LENGTH) & (pairs_df2['target'].str.len() <= MAX_SEQ_LENGTH)]
print('Number of Wikidata pairs:', len(pairs_df2), '\n')

pairs_df = pd.concat([pairs_df, pairs_df2]).sample(frac=1, random_state=RANDOM_STATE)
pairs_df

Number of JRC pairs: 131636
Number of Wikidata pairs: 434230 



,input,target
272102,katharine mccook knox,katherine mccook knox
431611,meri aroni,mary aroni
61867,alejandro foxley,alejandre foxley
424660,niche perez,limber perez
244085,jindrich wankel,heinrich wankel
...,...,...
440185,vitaly lisakovich,vital' lisakovic
122954,adam vojtech,adama vojtecha
27211,david petraeus,david petreaeus
164783,ethel standiford-mehling,ethel standiford-mehlingan


In [8]:
dce = DictionaryCharacterEncoder(max_seq_length=MAX_SEQ_LENGTH+2)
train_batches, val_batches = prepare_batches(pairs_df, dce, NUM_SAMPLES, VALIDATION_SPLIT, BATCH_SIZE, RANDOM_STATE)

2023-05-13 09:50:28.034541: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-13 09:50:28.034848: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-13 09:50:28.035028: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-13 09:50:28.660148: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-13 09:50:28.660329: I tensorflow/compile

In [9]:
for inputs, targets in train_batches.take(1):
    print(f'Encoder inputs shape: {inputs[0].shape}')
    print(f'Decoder inputs shape: {inputs[1].shape}')
    print(f'Targets shape: {targets.shape}')

Encoder inputs shape: (64, 32)
Decoder inputs shape: (64, 29)
Targets shape: (64, 29)


2023-05-13 09:50:40.904273: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype variant and shape [334150]
	 [[{{node Placeholder/_1}}]]
2023-05-13 09:50:40.904614: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype variant and shape [334150]
	 [[{{node Placeholder/_1}}]]


## Model

In [10]:
transformer = construct_model_w_teacher_forcing(
    num_encoder_layers=NUM_ENCODER_LAYERS, 
    num_decoder_layers=NUM_DECODER_LAYERS, 
    unique_tokens=len(dce.charset),
    max_seq_length=MAX_SEQ_LENGTH,
    embedding_dim=EMBEDDING_DIM,
    intermediate_dim=INTERMEDIATE_DIM,
    encoder_heads=NUM_ENCODER_HEADS,
    decoder_heads=NUM_DECODER_HEADS,
    dropout=DROPOUT
)

In [11]:
transformer.summary()
# transformer.compile("rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
transformer.compile(
    loss=masked_loss,
    optimizer=tf.keras.optimizers.Adam(),
    metrics=[masked_accuracy])

Model: "transformer_w_tf"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_inputs (InputLayer)    [(None, 32)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 32, 64)       2176        ['encoder_inputs[0][0]']         
                                                                                                  
 position_embedding (PositionEm  (None, 32, 64)      2048        ['embedding[0][0]']              
 bedding)                                                                                         
                                                                                                  
 tf.__operators__.add (TFOpLamb  (None, 32, 64)      0           ['embedding[0][0]'

## Training

In [12]:
steps_per_epoch = len(train_batches)

checkpoint_path = f'{model_serialization_path}/checkpoints/' + 'weights-{epoch:03d}'
checkpoint = tf.keras.callbacks.ModelCheckpoint(
                checkpoint_path, 
                monitor='val_masked_accuracy', 
                save_weights_only=True,
                save_freq=int(steps_per_epoch * CHECKPOINT_FREQ), 
                verbose=1)

In [13]:
history = transformer.fit(train_batches, epochs=EPOCHS, validation_data=val_batches, callbacks=[checkpoint])

Epoch 1/60


2023-05-13 09:50:42.827978: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype variant and shape [334150]
	 [[{{node Placeholder/_1}}]]
2023-05-13 09:50:42.828174: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype variant and shape [334150]
	 [[{{node Placeholder/_0}}]]
2023-05-13 09:50:49.819832: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-05-13 09:50:50.253416: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7f10683483c0 initialized for pl

5222/5222 [==============================] - ETA: 0s - loss: 0.9275 - masked_accuracy: 0.7655

2023-05-13 09:53:56.101583: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype variant and shape [165000]
	 [[{{node Placeholder/_1}}]]
2023-05-13 09:53:56.102120: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype variant and shape [165000]
	 [[{{node Placeholder/_1}}]]


5222/5222 [==============================] - 233s 42ms/step - loss: 0.9275 - masked_accuracy: 0.7655 - val_loss: 0.6136 - val_masked_accuracy: 0.8456
Epoch 2/60
5222/5222 [==============================] - 213s 41ms/step - loss: 0.6423 - masked_accuracy: 0.8368 - val_loss: 0.5589 - val_masked_accuracy: 0.8554
Epoch 3/60
5222/5222 [==============================] - 213s 41ms/step - loss: 0.5989 - masked_accuracy: 0.8454 - val_loss: 0.5309 - val_masked_accuracy: 0.8608
Epoch 4/60
5222/5222 [==============================] - 216s 41ms/step - loss: 0.5746 - masked_accuracy: 0.8501 - val_loss: 0.5138 - val_masked_accuracy: 0.8647
Epoch 5/60
5222/5222 [==============================] - 214s 41ms/step - loss: 0.5581 - masked_accuracy: 0.8532 - val_loss: 0.5014 - val_masked_accuracy: 0.8664
Epoch 6/60
5222/5222 [==============================] - 216s 41ms/step - loss: 0.5466 - masked_accuracy: 0.8556 - val_loss: 0.4963 - val_masked_accuracy: 0.8683
Epoch 7/60
5222/5222 [=======================

In [14]:
transformer.save(f'{model_serialization_path}/')

2023-05-13 13:20:04.436445: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,?,64]
	 [[{{node inputs}}]]
2023-05-13 13:20:04.445815: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'decoder_sequence' with dtype float and shape [?,?,64]
	 [[{{node decoder_sequence}}]]
2023-05-13 13:20:04.457185: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'decoder_sequence' with dtype float and shape [?,?,64]
	 [[{

INFO:tensorflow:Assets written to: /thesis/models/transformer_mixed_kerasnlp_1/assets


INFO:tensorflow:Assets written to: /thesis/models/transformer_mixed_kerasnlp_1/assets


In [15]:
with open(model_serialization_path + '/train_history.p', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

train_config = {
    'batch_size': BATCH_SIZE,
    'epochs': EPOCHS,
    'num_samples': NUM_SAMPLES,
    'max_seq_length': MAX_SEQ_LENGTH,
    'random_state': RANDOM_STATE,
    'validation_split': VALIDATION_SPLIT,
    'encoder_layers': NUM_ENCODER_LAYERS,
    'decoder_layers': NUM_DECODER_LAYERS,
    'encoder_heads': NUM_ENCODER_HEADS,
    'decoder_heads': NUM_DECODER_HEADS,
    'dropout': DROPOUT,
    'embedding_dim': EMBEDDING_DIM,
    'intermediate_dim': INTERMEDIATE_DIM
}

with open(model_serialization_path + '/config.p', 'wb') as file_pi:
    pickle.dump(train_config, file_pi)

In [16]:
transformer = tf.keras.models.load_model(f'{model_serialization_path}/', custom_objects={'masked_loss': masked_loss, 'masked_accuracy': masked_accuracy})

## Prediction

In [17]:
def decode_sequences(input_sentence):
     # Tokenize the encoder input.
    encoder_input_tokens = tf.keras.utils.pad_sequences(
        dce.to_ids([input_sentence], insert_markers=True), 
        padding='post', 
        maxlen=MAX_SEQ_LENGTH+2
    )

    # Define a function that outputs the next token's probability given the
    # input sequence.
    def token_probability_fn(decoder_input_tokens):
        return transformer([encoder_input_tokens, decoder_input_tokens])[:, -1, :]


    prompt = tf.fill((1, 1), dce.char_index['\t'])
    generated_tokens = keras_nlp.utils.top_p_search(
        token_probability_fn,
        prompt,
        p=0.1,
        max_length=MAX_SEQ_LENGTH,
        end_token_id=dce.char_index['\n'],
    )
    generated_sentences = ''.join([dce.inverse_char_index[tkn] for tkn in generated_tokens.numpy()[0]])
    return generated_sentences.strip()

names = [
    'samuel meyer',
    'dmitry medvedev',
    'paulo ricardo',
    'zouheir al qaissi',
    'tarek al bichri',
    'thorsten brotzmann'
]

for s in names:
    translated = decode_sequences(s)
    print(s)
    print(translated)
    print()


samuel meyer
samuel meyerova

dmitry medvedev
dmitrij medvedev

paulo ricardo
paulo ricardi

zouheir al qaissi
zuheir al qaissi

tarek al bichri
tarek al bichry

thorsten brotzmann
thorsten brotzman

